In [7]:
# simple genai app using langchain
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
# langchain tracking
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

loader = PyMuPDFLoader(file_path="Introduction_to_Stock_Markets.pdf")
document = loader.load()
doc_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = doc_splitter.split_documents(document)
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vectorstore_db = FAISS.from_documents(docs, embeddings)
retrievar = vectorstore_db.as_retriever()

llm = ChatOpenAI(model="gpt-4o")
prompt = ChatPromptTemplate.from_template(
    """
        Answer the question based only on the provided context:
        <context>
        {context}
        </context>
    """
)

document_chain = create_stuff_documents_chain(llm, prompt)
retrievar_chain = create_retrieval_chain(retrievar, document_chain)
print("="*60)
response1 = retrievar_chain.invoke({"input":"What is stock market?"})
print(response1['answer'])
print("="*60)
response2 = retrievar_chain.invoke({"input":"What is Bull Market?"})
print(response2['answer'])
print("="*60)
response3 = retrievar_chain.invoke({"input":"What is Face Value of a stock?"})
print(response3['answer'])
print("="*60)

What is the main purpose of the stock market according to the provided context?

The main purpose of the stock market is to facilitate transactions between buyers and sellers of shares.
Based on the provided context, what does it mean to be "bullish" on a stock price?

Being "bullish" on a stock price means that you believe the stock prices are likely to go up. In a broader perspective, if the stock market index is going up during a particular time period, it is referred to as a bull market.
What is the face value (FV) of a share, and how is it used in corporate actions?

The face value (FV) or par value of a share is the fixed denomination of a share. It is important for corporate actions such as dividends and stock splits. For example, dividends are often announced with the face value in perspective. If the face value of a stock is Rs. 5 and an annual dividend of Rs. 63 is declared, the dividend yield would be 1260% (63 divided by 5). In stock splits, the face value can change, resul